<a href="https://colab.research.google.com/github/kumaranil02/NLP_TensorFlow/blob/main/LLMas/Llama2_finetuning_on_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Necessary Libraries

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9

# Dataset details
Instacart Data can be downloaded from [here](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data). We just need product & department csv files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/'Colab Notebooks'


'Abhilash_question 6.ipynb'
'Amazon_analysis_using_python - shruti'
'Amazon reviews dataset_analysis.ipynb'
'Anil_Experiments_management_with_Weights_&_Biases.ipynb'
'Answer 1 .ipynb'
'Answer 3.ipynb'
'Answer 5.ipynb'
'Answer 6.ipynb'
 Cluster_Analysis.ipynb
 container-with-most-water.ipynb
'Copy of C4W3_Colab_T5_SQuAD_Model.ipynb'
'Copy of Experiments management with Weights & Biases.ipynb'
'Copy of Falcon finetuning on Instacart Dataset'
'Copy of WandB_Prompts_Quickstart (1).ipynb'
'Copy of WandB_Prompts_Quickstart.ipynb'
 CreditCardAnalysis.ipynb
 Eye_for_blind_Starter_code__check.ipynb
 HuggingFace_BERT_SQuAD_Finetuning.ipynb
 LargeDataScrap_Selenium_17M_.ipynb
 Linear_regression_question_1_MA-2.ipynb
 Linear+SVM+-+Email+Spam+Classifier.ipynb
'Llama2 finetuning on Custom Dataset'
 LLM_Falcon
 P_ANIL_KUMAR_CAPSTONE_SUBMISSION.ipynb
 Project_Sentiment_Analysis_using_BERT.ipynb
 Q5.ipynb
'Question 5 .pynb file .ipynb'
 sales_prediction_simple_linear_regression.ipynb
 spacy_train.ipynb

In [5]:
import pandas as pd
df_product = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LLM_Falcon/products.csv")
df_dept = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LLM_Falcon/departments.csv')

In [6]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])
df_joined['text'] = df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis = 1)

In [7]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=0.2, random_state=42)

In [8]:
train_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...
18377,29407,Root Vegetable Cakes,42,1,frozen,Root Vegetable Cakes ->: frozen
28627,29571,Alta Dena 1% Milk,84,16,dairy eggs,Alta Dena 1% Milk ->: dairy eggs
30283,3589,Laundry Detergent Pods,75,17,household,Laundry Detergent Pods ->: household
32030,31685,"Daily Shower Cleaner Scrub Free Refill, Fresh ...",114,17,household,"Daily Shower Cleaner Scrub Free Refill, Fresh ..."
28655,30119,Organic French Style Meyer Lemon Yogurt,120,16,dairy eggs,Organic French Style Meyer Lemon Yogurt ->: da...


In [9]:
test_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...
31573,24121,Dust Pan,114,17,household,Dust Pan ->: household
45362,5477,Roasted Pine Nut Hommus,67,20,deli,Roasted Pine Nut Hommus ->: deli
14131,27921,Cranberry Raspberry Juice Cocktail,98,7,beverages,Cranberry Raspberry Juice Cocktail ->: beverages
26903,4786,Sweet Cream Butter Salted,36,16,dairy eggs,Sweet Cream Butter Salted ->: dairy eggs
39417,8796,Traditional Chicken Barley Soup,69,15,canned goods,Traditional Chicken Barley Soup ->: canned goods


In [10]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

**Let's check what the base model predicts before finetuning. :)**

In [13]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Result: “Free & Clear Stage 4 Overnight Diapers” ->:
 Unterscheidung:
 - 2400
 - 7248
Darstellende Personen (Kopie):
- 7955
Darstellende Personen (Kopie):
- 7956
Darstellende Personen (Kopie):
- 9916
 Differenz 0,0000000000 (0):
\end{code}

Comment: Thanks a lot. This is a perfect answer. I was looking for a solution to this problem for a while and finally found it here!
Result: Bread Rolls ->:
 Hinweis: Die Rechnung für 1 Kilo Bread Rolls ist nur für 1 kg. 2, 3 kg etc.
Haben Sie eine Anfrage an "1 Kilo Bread Rolls"?
Result: French Milled Oval Almond Gourmande Soap ->: 1. февруари.
Milk, Water, Almonds, Glycerin, Palmitic Acid, Stearic Acid, Lactic Acid, Potassium Sorbate, Sodium Hydroxide, Titanium Dioxide, Yellow 5.


Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [14]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [15]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [16]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:119: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [17]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [18]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.029100
2,3.089600
3,4.359200
4,6.271200
5,4.250100
6,3.091500
7,4.516300
8,3.965800
9,4.466900
10,5.350400


TrainOutput(global_step=120, training_loss=2.6901778891682624, metrics={'train_runtime': 488.902, 'train_samples_per_second': 3.927, 'train_steps_per_second': 0.245, 'total_flos': 627303342243840.0, 'train_loss': 2.6901778891682624, 'epoch': 0.05})

In [19]:
lst_test_data = list(test_df['text'])

In [20]:
len(lst_test_data)

9938

In [21]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [23]:
data_test = []
for i in lst_test_data_short:
  data_test.append(i.split(':')[0])

In [24]:
data_test

['Free & Clear Stage 4 Overnight Diapers ->',
 'Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->',
 'Coffee Liquer ->',
 'Bread Rolls ->',
 'French Milled Oval Almond Gourmande Soap ->',
 'Dust Pan ->',
 'Roasted Pine Nut Hommus ->',
 'Cranberry Raspberry Juice Cocktail ->',
 'Sweet Cream Butter Salted ->',
 'Traditional Chicken Barley Soup ->',
 'Vanilla Unsweetened Cashewmilk ->',
 'Minis Size Chocolate Candy Bars Variety Mix ->',
 'Cheesy Cheddar Rotini Pasta Sides ->',
 'Multi Purpose Solution ->',
 'Juice, Raw & Cold-Pressed, Purity ->',
 'Chinese Five Spices Powder ->',
 'Ancient Grain Original Granola ->',
 'Crunch Lemon Shortbread Flavor 0% Fat with Toppings Greek Yogurt ->',
 'Flounder Fillets ->',
 'Lilac Votive ->',
 'Cheese & Broccoli Ravioli For Kids ->',
 'Organic Soymilk Original Value Size ->',
 'Fruit Juice Sweetened Millet Rice Cereal ->',
 'Morning Burst® Facial Scrub Cleansers ->',
 'Salmon Pinwheels ->']

In [25]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    data_test,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0 Free & Clear Stage 4 Overnight Diapers ->: baby care personal care baby wipes stage 4 diaper pads ->: baby care diapers & wipes diapers ->: baby care baby wipes wipes ->: personal care baby wipes wipe clean baby wipes free & clear stage 2 diaper pads ->: personal care diapers & wipes personal wipes personal wipers ->: diapers & wipes diapers free clean: diaper
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: international beverages beer, wine, liquor ->: beverage alcohol beverages ->: wine ->: beer, wine, liquor international beverages beer, wine, liquor ->: beverage alcohol beverages ->: wine ->: beer, wine, liquor b
2 Coffee Liquer ->: beverages juices coffee creamers coffee coffee liquer ->: beverages coffee creamers coffee coffee liquer coffee creme de menthe ->: snacks & beverages beverages coffee creamers coffee coffee liquer coffee creme de menthe ->: snacks & beverages beverages drinks non-alcoholic ->: snacks & b

In [26]:
lst_test_data_short

['Free & Clear Stage 4 Overnight Diapers ->: babies',
 'Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen',
 'Coffee Liquer ->: alcohol',
 'Bread Rolls ->: bakery',
 'French Milled Oval Almond Gourmande Soap ->: personal care',
 'Dust Pan ->: household',
 'Roasted Pine Nut Hommus ->: deli',
 'Cranberry Raspberry Juice Cocktail ->: beverages',
 'Sweet Cream Butter Salted ->: dairy eggs',
 'Traditional Chicken Barley Soup ->: canned goods',
 'Vanilla Unsweetened Cashewmilk ->: dairy eggs',
 'Minis Size Chocolate Candy Bars Variety Mix ->: snacks',
 'Cheesy Cheddar Rotini Pasta Sides ->: dry goods pasta',
 'Multi Purpose Solution ->: personal care',
 'Juice, Raw & Cold-Pressed, Purity ->: beverages',
 'Chinese Five Spices Powder ->: international',
 'Ancient Grain Original Granola ->: breakfast',
 'Crunch Lemon Shortbread Flavor 0% Fat with Toppings Greek Yogurt ->: dairy eggs',
 'Flounder Fillets ->: meat seafood',
 'Lila

In [27]:
a=': baby care personal care baby wipes stage 4 diaper pads ->: baby care diapers & wipes diapers ->: baby care baby wipes wipes ->: personal care baby wipes wipe clean baby wipes free & clear stage 2 diaper pads ->: personal care diapers & wipes personal wipes personal wipers ->: diapers & wipes diapers free clean: diaper'

In [29]:
len(a.split())

58

In [31]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    data_test,
    max_length=50,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

0 Free & Clear Stage 4 Overnight Diapers ->: personal care baby care diapers ->: personal hygiene toiletries ->: baby care baby care diapers ->: diapers ->: diaper products & accessories stage
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: cans & bottles canned food ->: meat, fish, p
2 Coffee Liquer ->: pantry & household supplies beverages coffee beverages ->: drinks coffee drinks ->: espresso coffee drinks ->: espresso coffee beverages: espresso liquor ->
3 Bread Rolls -> snacks -> bread & breakfast -> frozen -> frozen breads and rolls -> frozen rolls -> frozen bread rolls -> Frozen Bread Rolls French Bread Rolls -> grocery ->
4 French Milled Oval Almond Gourmande Soap ->: personal care household personal care body care soap ->: soaps body care soaps bar & liquid soaps ->: soaps & body care products bath, hair care, hand lot
5 Dust Pan ->: household cleaning floor cleaning mop pads ->: household cleaning cleaning suppli

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies: diapers & accessories diapers',
 'frozen goods',
 'alcoholic beverages spirits',
 'bakery',
 'personal care soap & bath',
 'household: pantry: cleaning: broom & mop: broom & broom brushes: hand broom',
 'deli meats & cheese',
 'beverages alcohol juice cranberry raspberry juice cocktail',
 'dairy eggs butter & cheese eggs',
 'canned goods',
 'dairy eggs & dairy eggs',
 'snacks chocolate & candy',
 'dry goods pasta, rice, beans and grains pasta',
 'personal care toiletries personal care toiletries liquid hand wash & dish wash',
 'beverages',
 'international foods dry goods spices & seasonings spices five spices',
 'breakfast cereals',
 'dairy eggs & cheese frozen dairy',
 'meat seafood frozen',
 'household essentials candles & accessories scented votive wax tablets',
 'frozen food',
 'dairy eggs dairy egg products soy products',
 'breakfast foods',
 'personal care',
 'meat seafood fish seafood']

In [ ]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies: diapers & accessories diapers
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen goods
2,Coffee Liquer,alcohol,alcoholic beverages spirits
3,Bread Rolls,bakery,bakery
4,French Milled Oval Almond Gourmande Soap,personal care,personal care soap & bath
5,Dust Pan,household,household: pantry: cleaning: broom & mop: broo...
6,Roasted Pine Nut Hommus,deli,deli meats & cheese
7,Cranberry Raspberry Juice Cocktail,beverages,beverages alcohol juice cranberry raspberry ju...
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs butter & cheese eggs
9,Traditional Chicken Barley Soup,canned goods,canned goods
